## 0. Import Required Packages

In [153]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

## 1. Data Wrangling

In [145]:
data = pd.read_csv('./research_dataset_2022.csv', parse_dates=True, index_col=0)
# delete the rows before 1993-02-02
data.dropna(subset=['trans_spy_open'], inplace=True)
returns = data[['trans_spy_open']]

### 1.1 Process variables referenced Hull and Qiao's research [2017]

In [86]:
var = data[['DividendToPriceRatio', 'trans_PE_RATIO', 'PxToBook', 'CAPE', 
            'bbyield', 'BAA', 'AAA', 'US3M', 'US10YR', 'Consumption_new', 
            'Assets_new', 'Income_new', 'SIM', 'trans_VIX', 'garch', 'SI',
            'IC', 'trans_BDIY', 'NOS', 'CPI', 'trans_spy_open', 'trans_SPGSCI']]
rest_data = data.drop(columns=var.columns.tolist())

In [87]:
var['DEF'] = var['BAA'] - var['AAA']
var['TERM'] = var['US10YR'] - var['US3M']
#var['CAY'] = 
var['VRP'] = var['trans_VIX'] - var['garch']
var['PCR'] = var['trans_spy_open'] / var['trans_SPGSCI']
# drop the component variables
var = var.drop(columns=['BAA', 'AAA', 'US10YR', 'US3M', 'trans_VIX', 'garch', 
                        'trans_spy_open', 'trans_SPGSCI'])
var.rename(columns = {'DividendToPriceRatio':'DP', 'trans_PE_RATIO':'PE',
                      'PxToBook':'BM', 'bbyield':'BY', 'trans_BDIY':'BDI'}, inplace = True)

### 1.2 Calculate the response variable

In [158]:
def calculate_returns(ndays):
    returns[str(ndays)+'return'] = np.nan
    for i in range(len(returns)):
        if i-ndays >= 0:
            present = returns.iloc[i]['trans_spy_open']
            past = returns.iloc[i-ndays]['trans_spy_open']
            returns[str(ndays)+'return'].iloc[i] = (present - past) / past
        else:
            pass

In [159]:
# 1-month return
calculate_returns(22)
# 3-month return
calculate_returns(65)
# 6-month return
calculate_returns(130)
# 1-year return
calculate_returns(250)

In [160]:
returns.isna().sum()

trans_spy_open       0
22return            22
65return            65
130return          130
250return          250
a                 7309
dtype: int64